# Main Romain

## Ideas

- A lot of unsuccesfull execution of task, might indicates 'trying to find a breach'

## Library Loading

In [2]:
## Classical libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
## Specific libraries

In [4]:
from collections import Counter
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

## Loading the data

In [5]:
path = 'Data/train/0a8e69f80f39b18a78ca7B778a4efb029e7b42fbf.Zbot.xml'
tree = ET.parse(path)
root = tree.getroot()

## Features Engineering

In [23]:
def XML_get(root, element, attrib_name=''):
    res = []
    if attrib_name != '':
        for t in root.iter(element):
            try: 
                res.append(t.attrib[attrib_name])
            except:
                res.append(None)
    else:
        for t in root.iter(element):
            try:
                res.append(t.attrib)
            except:
                res.append(None)
    return res

### Get some potential features

    # Load DLL
load_dll_files              = XML_get(root, 'load_dll',   'filename')

    # VM_Protect
vm_protect_target           = XML_get(root, 'vm_protect', 'target')
vm_protect_protect          = XML_get(root, 'vm_protect', 'protect')
vm_protect_behavior         = XML_get(root, 'vm_protect', 'behavior')

    # Open Key
open_key_key                = XML_get(root, 'open_key', 'key')

    # Process
process_filename            = XML_get(root, 'process', 'filename')
process_filesize            = XML_get(root, 'process', 'filesize')
process_username            = XML_get(root, 'process', 'username')
process_applicationtype     = XML_get(root, 'process', 'applicationtype')
process_terminationreason   = XML_get(root, 'process', 'terminationreason')

    # Set Files Attributes
set_file_attributes_srcfile = XML_get(root, 'set_file_attributes', 'srcfile')

    # Open File
open_file_filetype      = XML_get(root, 'open_file', 'filetype')
open_file_srcfile       = XML_get(root, 'open_file', 'srcfile')
open_file_desiredaccess = XML_get(root, 'open_file', 'desiredaccess')

    # Find the successes
successes = []
for t in root.findall(".//*[@successful]"):
    successes.append(int(t.attrib['successful']))
success_ratio = sum(successes) / float(len(successes))




In [24]:
load_dll_files

['C:\\WINDOWS\\system32\\ntdll.dll',
 'C:\\WINDOWS\\system32\\kernel32.dll',
 'C:\\WINDOWS\\system32\\USER32.dll',
 'C:\\WINDOWS\\system32\\GDI32.dll',
 'C:\\WINDOWS\\system32\\msvcrt.dll',
 'C:\\WINDOWS\\system32\\ole32.dll',
 'C:\\WINDOWS\\system32\\ADVAPI32.dll',
 'C:\\WINDOWS\\system32\\RPCRT4.dll',
 'C:\\WINDOWS\\system32\\Secur32.dll',
 'C:\\WINDOWS\\system32\\IMM32.DLL',
 'C:\\WINDOWS\\system32\\oleaut32.dll',
 'C:\\WINDOWS\\system32\\pstorec.dll',
 'C:\\WINDOWS\\system32\\ATL.DLL',
 'C:\\WINDOWS\\system32\\SHLWAPI.dll',
 'C:\\WINDOWS\\system32\\kernel32.dll',
 'C:\\WINDOWS\\system32\\IMM32.DLL',
 'C:\\WINDOWS\\system32\\shell32.dll',
 'C:\\WINDOWS\\system32\\ntdll.dll',
 'C:\\WINDOWS\\system32\\SHLWAPI.dll',
 'C:\\WINDOWS\\system32\\psapi.dll',
 'C:\\WINDOWS\\system32\\wininet.dll',
 'C:\\WINDOWS\\system32\\ADVAPI32.dll',
 'C:\\WINDOWS\\system32\\ws2_32.dll',
 'C:\\WINDOWS\\system32\\wsock32.dll',
 'C:\\WINDOWS\\system32\\USER32.dll',
 'C:\\WINDOWS\\system32\\ole32.dll',
 'C:\\

##### Find all the tags 

In [99]:
all_tags = []
for t in root.iter('all_section'):
    for child in t.getchildren():
        all_tags.append(child.tag)


In [101]:
set(all_tags)

{'bind_socket',
 'check_for_debugger',
 'com_create_instance',
 'connect',
 'copy_file',
 'create_interface',
 'create_key',
 'create_mutex',
 'create_open_file',
 'create_socket',
 'create_thread',
 'create_thread_remote',
 'delete_file',
 'enum_keys',
 'enum_modules',
 'enum_processes',
 'enum_types',
 'enum_values',
 'find_file',
 'get_computer_name',
 'get_file_attributes',
 'get_host_by_name',
 'get_system_directory',
 'get_system_time',
 'get_username',
 'impersonate_user',
 'kill_process',
 'listen_socket',
 'load_dll',
 'load_image',
 'move_file',
 'open_file',
 'open_key',
 'open_mutex',
 'open_process',
 'open_scmanager',
 'open_service',
 'open_url',
 'query_value',
 'revert_to_self',
 'set_file_attributes',
 'set_file_time',
 'set_value',
 'set_windows_hook',
 'sleep',
 'vm_allocate',
 'vm_protect',
 'vm_write'}